In [1]:
# could use GPU if run the following command
#   pip install tensorflow-gpu
#
import tensorflow as tf
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

with tf.Session() as sess:
        x.initializer.run()
        y.initializer.run()
        result = f.eval()
        print(result)

print(f)

42


In [3]:
# Running Tensort flow with the use of Perceptrons
from sklearn import datasets
import numpy as np
import tensorflow as tf

digits = datasets.load_digits()


x = tf.placeholder(tf.float32, [None, 64])

W = tf.Variable(tf.zeros([64, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x, W) + b)

y_ = tf.placeholder(tf.float32, [None, 10])
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
cross_entropy = -tf.reduce_sum(y_*tf.log(tf.clip_by_value(y,1e-10,1.0)))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')

init = tf.initialize_all_variables()

saver = tf.train.Saver()

sess = tf.Session()
sess.run(init)

dense_target = np.zeros([digits.target.size, 10])
for i in range(digits.target.size):
    dense_target[i, digits.target[i]] = 1

for step in range(10):
    batch_xs = digits.data
    batch_ys = dense_target
    sess_output = sess.run([train_step, loss, W], feed_dict={x: batch_xs, y_: batch_ys})
    print('Step %d: loss = %.2f' % (step, sess_output[1]))

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: digits.data[1:100], y_: dense_target[1:100]}))
saver.save(sess, 'perceptron', global_step=step)


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Step 0: loss = 4137.77
Step 1: loss = 4810.30
Step 2: loss = 4443.99
Step 3: loss = 4443.99
Step 4: loss = 4443.99
Step 5: loss = 4443.99
Step 6: loss = 4443.99
Step 7: loss = 4443.99
Step 8: loss = 4443.99
Step 9: loss = 4443.99
0.868687


'perceptron-9'

In [6]:
# Running tensorflow using a Convolutional Neural Network

from sklearn import datasets
import numpy as np
import tensorflow as tf

sess = tf.InteractiveSession()

digits = datasets.load_digits()

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])  # 5x5 grid
b_conv1 = bias_variable([32])

x = tf.placeholder(tf.float32, shape=[None, 64])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.reshape(x, [-1,8,8,1]) # fit data into a tensor

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([2 * 2 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 2*2*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])



y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())

dense_target = np.zeros([digits.target.size, 10])
for i in range(digits.target.size):
    dense_target[i, digits.target[i]] = 1


saver = tf.train.Saver()


for i in range(100):
  batch_idxs = np.random.randint(1,1000, 50)

  batch_xs = digits.data[batch_idxs]
  batch_ys = dense_target[batch_idxs]



  if i%10 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch_xs, y_: batch_ys, keep_prob: 1.0})

    print("step %d, training accuracy %g"%(i, train_accuracy))

    test_accuracy = accuracy.eval(feed_dict={
            x:digits.data[1000:1100], y_: dense_target[1000:1100], keep_prob: 1.0})

    print("test accuracy %g"%(test_accuracy))


  train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})

#correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

test_accuracy = accuracy.eval(feed_dict={
        x:digits.data[1000:1100], y_: dense_target[1000:1100], keep_prob: 1.0})

print("test accuracy %g"%(test_accuracy))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, training accuracy 0.1
test accuracy 0.08
step 10, training accuracy 0.24
test accuracy 0.08
step 20, training accuracy 0.3
test accuracy 0.27
step 30, training accuracy 0.5
test accuracy 0.41
step 40, training accuracy 0.52
test accuracy 0.46
step 50, training accuracy 0.7
test accuracy 0.59
step 60, training accuracy 0.82
test accuracy 0.71
step 70, training accuracy 0.86
test accuracy 0.71
step 80, training accuracy 0.9
test accuracy 0.81
step 90, training accuracy 0.88
test accuracy 0.82
test accuracy 0.81


ImportError: No module named cv2

In [ ]:
# Running the resnet model
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

In [1]:
# Recognizing an elephant
img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
model.summary()
preds = model.predict(x)

print('Predicted:', decode_predictions(preds, top=3)[0])

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

('Predicted:', [(u'n01871265', u'tusker', 0.41353554), (u'n02504013', u'Indian_elephant', 0.4001196), (u'n02504458', u'African_elephant', 0.18621385)])


In [2]:
# Recognizing a parrot
img_path = 'a_parrot.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
model.summary()
preds = model.predict(x)

print('Predicted:', decode_predictions(preds, top=3)[0])


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

('Predicted:', [(u'n01817953', u'African_grey', 0.99972898), (u'n01818515', u'macaw', 7.564493e-05), (u'n01855672', u'goose', 5.90854e-05)])
